In [ ]:
#clear all
%reset -f

#import packages
import numpy as np
import scipy
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [ ]:
# Analyses are similar to FRN, but with different electrodes
# First, we get the 800 timepoints we are considering as a list, to use for indices given by cluster-based permutation later
# But only grab 0 to 1 sec time-locked to feedback onset (idx= 400:601)
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')
pp = 0 #only need one participant

# we can use aligned data

id_directory = os.path.join(data_directory, 'p%03d/' % pp)
pp_directory = os.path.join(id_directory, erps)
fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (pp, 'SmallLarge', 'P3', 'SL', 'aligned'))
evoked = mne.read_evokeds(fname)
df = evoked[0].to_data_frame()
time = df['time'].tolist()
time = time[400:601] #get only timepoints we want

In [ ]:
# Load data for each condition
# Take the mean across electrodes of interest per participant
# transform data into array accepted for cluster-based permutation test in mne
# output: (n_participants, n_timepts) for each condition

root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#specify channels we need - P3
channels = ['FCz', 'Fz', 'Cz', 'Pz']

    
#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = ['sml', 'lrg']


#aligned
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (participants[pp], 'SmallLarge', 'P3', 'SL', 'aligned'))
    evoked = mne.read_evokeds(fname)
    evoked = evoked[0]
    evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
    evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
    evoked = evoked[400:601] #get timepoints we need
    evoked_list.append(evoked)
    
aligned_flist = evoked_list

#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s_%s_%s-ave.fif' % (participants[pp], 'SmallLarge', 'P3', conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            evoked = evoked[0]
            evoked = evoked.get_data(picks=channels) #will give data of shape (n_channels, n_timepts)
            evoked = evoked.mean(axis=0) #take mean of cols or the channels we picked
            evoked = evoked[400:601] #get timepoints we need
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            small_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            large_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            small_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            large_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            small_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            large_mir_flist = evoked_list

In [ ]:
# Comparing two ERP signals is just the same as taking their difference (ERP1 minus ERP2) and using this in permutation test
def get_clust_perm_test(conditionA, conditionB, pval, n_permutations):
    #take difference of two conditions
    data = np.subtract(conditionA, conditionB)
#     np.shape(data)
    #define cluster forming threshold based on p-value
    df = len(participants) - 1
    thresh = scipy.stats.t.ppf(1 - pval / 2, df)
    #run cluster-based permutation test
    T_0, clust_idx, clust_pvals, H0 = mne.stats.permutation_cluster_1samp_test(data, threshold = thresh, 
                                                          n_permutations = n_permutations, tail = 0, 
                                                          adjacency = None, seed = 999, 
                                                          out_type = 'mask', verbose = True)

    return T_0, clust_idx, clust_pvals, H0

In [ ]:
# First, we can compare each condition to aligned
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values
# This information can then be included in ERP plots

flists = [small_rot_flist, large_rot_flist, small_rdm_flist, large_rdm_flist, small_mir_flist, large_mir_flist]
conditionnames = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for f in range(0, len(flists)):
    T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(flists[f], aligned_flist, p, perms)
#     print(clust_idx, clust_pvals)
    if len(clust_idx) == 0:
        condition.append(conditionnames[f])
        clust_idx_start.append(np.nan)
        clust_idx_end.append(np.nan)
        time_start.append(np.nan)
        time_end.append(np.nan)
        p_values.append(np.nan)
    else:
        for clust in range(0, len(clust_idx)):
            cluster = clust_idx[clust][0] #to get the slice sequence we need
        
            cluster_start = cluster.start
            clust_idx_start.append(cluster_start)
        
            cluster_end = cluster.stop
            clust_idx_end.append(cluster_end)
        
            time_idx_start = time[cluster_start]
            time_start.append(time_idx_start)
        
            time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
            time_end.append(time_idx_end)
        
            clust_p = clust_pvals[clust]
            p_values.append(clust_p)
        
            condition.append(conditionnames[f])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_vsAligned_SmallLarge_P3.csv')
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next, we subtract aligned from each condition, so that we can compare small vs large in each perturbation type
diffconds = ['smallrot', 'largerot', 'smallrdm', 'largerdm', 'smallmir', 'largemir']
smallrot_diff = []
largerot_diff = []
smallrdm_diff = []
largerdm_diff = []
smallmir_diff = []
largemir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(small_rot_flist, aligned_flist)
        smallrot_diff.append(diffevks)
        smallrot_diff = smallrot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(large_rot_flist, aligned_flist)
        largerot_diff.append(diffevks)
        largerot_diff = largerot_diff[0]
    elif cond == 2:
        diffevks = np.subtract(small_rdm_flist, aligned_flist)
        smallrdm_diff.append(diffevks)
        smallrdm_diff = smallrdm_diff[0]
    elif cond == 3:
        diffevks = np.subtract(large_rdm_flist, aligned_flist)
        largerdm_diff.append(diffevks)
        largerdm_diff = largerdm_diff[0]
    elif cond == 4:
        diffevks = np.subtract(small_mir_flist, aligned_flist)
        smallmir_diff.append(diffevks)
        smallmir_diff = smallmir_diff[0]
    elif cond == 5:
        diffevks = np.subtract(large_mir_flist, aligned_flist)
        largemir_diff.append(diffevks)
        largemir_diff =largemir_diff[0]

In [ ]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of SMALL VS LARGE comparisons
# This information can then be included in ERP difference wave plots

conds = ['rot_diff', 'rdm_diff', 'mir_diff']
conditionnames = ['rot', 'rdm', 'mir']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largerot_diff, smallrot_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largerdm_diff, smallrdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(largemir_diff, smallmir_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:    
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_SmallvsLarge_P3.csv')
perm_test.to_csv(perm_test_filename)

In [ ]:
# Next step is to subtract early from late condition, to generate a single signal for each perturbation
diffconds = ['rot', 'rdm', 'mir']
rot_diff = []
rdm_diff = []
mir_diff = []

for cond in range(0, len(diffconds)):
    if cond == 0:
        diffevks = np.subtract(largerot_diff, smallrot_diff)
        rot_diff.append(diffevks)
        rot_diff = rot_diff[0] #to keep shape of object consistent
    elif cond == 1:
        diffevks = np.subtract(largerdm_diff, smallrdm_diff)
        rdm_diff.append(diffevks)
        rdm_diff = rdm_diff[0]
    elif cond == 2:
        diffevks = np.subtract(largemir_diff, smallmir_diff)
        mir_diff.append(diffevks)
        mir_diff = mir_diff[0]

In [ ]:
# Generate a data frame to tabulate condition, cluster indices, cluster timepts, p values of PERTURBATION comparisons
# This information can then be included in ERP difference wave plots

conds = ['rotvmir', 'rotvrdm', 'mirvrdm']
conditionnames = ['rotvmir', 'rotvrdm', 'mirvrdm']
p = 0.05
perms = 1000

condition = []
clust_idx_start = []
clust_idx_end = []
time_start = []
time_end = []
p_values = []

for c in range(0, len(conds)):
    if c == 0:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, mir_diff, p, perms)
#         print(clust_idx, clust_pvals)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 1:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(rot_diff, rdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
            
    elif c == 2:
        T_0, clust_idx, clust_pvals, H0 = get_clust_perm_test(mir_diff, rdm_diff, p, perms)
        if len(clust_idx) == 0:
            condition.append(conditionnames[c])
            clust_idx_start.append(np.nan)
            clust_idx_end.append(np.nan)
            time_start.append(np.nan)
            time_end.append(np.nan)
            p_values.append(np.nan)
        else:
            for clust in range(0, len(clust_idx)):
                cluster = clust_idx[clust][0] #to get the slice sequence we need
        
                cluster_start = cluster.start
                clust_idx_start.append(cluster_start)
        
                cluster_end = cluster.stop
                clust_idx_end.append(cluster_end)
        
                time_idx_start = time[cluster_start]
                time_start.append(time_idx_start)
        
                time_idx_end = time[cluster_end - 1] #minus one because python indexing does not include ending value
                time_end.append(time_idx_end)
        
                clust_p = clust_pvals[clust]
                p_values.append(clust_p)
        
                condition.append(conditionnames[c])
        
perm_test = pd.DataFrame(
    {'condition': condition,
     'clust_idx_start': clust_idx_start,
     'clust_idx_end': clust_idx_end,
     'time_start': time_start,
     'time_end': time_end,
     'p_values': p_values})

perm_test_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Permutation_test_SmallLarge_PerturbTypeComp_P3.csv')
perm_test.to_csv(perm_test_filename)